# Konfigurasi Awal
Bagian konfigurasi awal untuk mendapatkan file input,
## Download from google cloud storage
Semua file input maupun output disimpan di cloud storage
### Nama file
file input: data/input-latest.csv
file output: data/output-latest.json
> Boleh juga jika modelnya ingin diexport
#### File input dikoleksi setiap hari dengan format input-timestamp.csv

In [ ]:
import os
import glob
import pandas as pd
from datetime import datetime

In [ ]:
import sys
!{sys.executable} -m pip install python-dotenv

In [ ]:
from dotenv import load_dotenv

load_dotenv() 
os.environ["TFVERSION"] = "2.5"
os.environ["PYTHONVERSION"] = "3.7"

In [ ]:
%%bash

mkdir -p data
gsutil -q -m cp gs://$BUCKET_NAME/data/input* data/

In [ ]:
inputList = sorted(glob.glob('data/input-1*.csv'))
print(inputList)
isOk = glob.glob('data/input-aafsa.csv')
not inputList

In [ ]:
def read_csv_add_date(path, isNeedParse=True):
    csvData = pd.read_csv(path)
    if isNeedParse:
        csvData["date"] = pd.to_datetime(
            datetime.fromtimestamp(
                int(path.split("-")[-1][:-4])//1000
            ))
    return csvData

In [ ]:
def preprocess_merge():
    isMergedDatasetExist = glob.glob('data/input-preprocessed.csv')
    listOfDatasets = sorted(glob.glob('data/input-1*.csv'))
    if not isMergedDatasetExist:
        df_merge = pd.DataFrame()
        for i, dataset in enumerate(listOfDatasets[:-1]):
            df1 = read_csv_add_date(listOfDatasets[i])
            df2 = read_csv_add_date(listOfDatasets[i+1])
            df2["visitor"] = df2["visitor"].sub(df1["visitor"])
            if (i == 0):
                df_merge = df_merge.append(df1, ignore_index = True)
            df_merge = df_merge.append(df2, ignore_index = True)
            df_merge.head(10);
            
        df_merge = df_merge.sort_values(
            ["id", "date"], ascending = (True, True))
        df_merge["date"] = pd.to_datetime(
            df_merge.date).dt.strftime("%-d/%-m/%Y %-H:%M")
        
        df_merge.to_csv("data/input-preprocessed.csv", index=False)
#         display(df_merge.head())
        
    else:
        df_merge = read_csv_add_date('data/input-preprocessed.csv', isNeedParse=False)
        df_merge["date"] = pd.to_datetime(
            df_merge.date)
        
        df_to_merge = read_csv_add_date(
            listOfDatasets[-1]) # read previous cumulative dump
        df_to_sub = read_csv_add_date(
            listOfDatasets[-2]) # read previous 2 cumulative dump
        
        df_to_merge["visitor"] = df_to_merge["visitor"].sub(df_to_sub["visitor"]) # get non-cumulative visitor sum
        
        df_merge = df_merge.append(
            df_to_merge, ignore_index = True)
        df_merge = df_merge.sort_values(
            ["id", "date"], ascending = (True, True))
        
        df_merge["date"] = pd.to_datetime(
            df_merge.date).dt.strftime("%-d/%-m/%Y %-H:%M")
        df_merge.to_csv("data/input-preprocessed.csv", index=False)
#         display(df_merge.head())

In [ ]:
preprocess_merge()

In [ ]:
listOfDatasets = sorted(glob.glob('data/input-1*.csv'))
display(listOfDatasets)
display(listOfDatasets[:-1])


In [ ]:
len(df_merge)
df_merge = df_merge.sort_values(["id", "date"], ascending = (True, False));

In [ ]:
df_merge.head(10)

In [ ]:
df_merge.to_csv("data/input-raw.csv", index=False)

### Upload file output
File output diupload ke cloud storage

#### Format file output: 
`output-latest.json` dan `output-[timestamp].json`, boleh juga kalau mau upload bentuk lainnya (misal HDF5 nya dll.)
> Pastikan output **disimpan ke direktori data**

In [ ]:
%%bash

gsutil cp data/output* gs://$BUCKET_NAME/data/